# DataSport Scraping

In [ ]:
import requests as rq
from bs4 import BeautifulSoup as bfs
from IPython.display import HTML
import pandas as pd
import numpy as np
import collections

## Scraping a table

In [ ]:
HTML('<iframe src=https://services.datasport.com/2015/lauf/transviamala/alfaw.htm width=1000 height=350></iframe>')

In [ ]:
page = rq.get('https://services.datasport.com/2015/lauf/transviamala/alfaw.htm')
soup = bfs(page.text, 'html5lib')

fonts = soup.select('pre > font')
data = fonts[2].text.split('\n')

In [ ]:
data_pd = pd.DataFrame(data)
data_pd['length'] = data_pd.apply(lambda x: len(x[0]), axis=1)
data_pd['length'].value_counts()

In [ ]:
max_line = max(data, key=len)
clean_data = [line for line in data if len(line) > 10]

In [ ]:
lines = []
columns = []
for i in range(0, len(max_line)):
    nb_blank = 0
    for j, line in enumerate(clean_data):
        if len(line) > i:
            if line[i] == ' ':
                nb_blank += 1
            else:
                if len(lines) == 0:
                    lines = ['']*len(clean_data)
                lines[j] += line[i]
    if nb_blank == len(clean_data) and len(lines):
        columns.append(lines)
        lines = []

pd.DataFrame(columns).T